#**Limpeza de Banco de Dados dos Bombeiros de Joinville com regex**

Monta o link com google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Instala biblioteca Fuzzywuzzy

In [ ]:
#instala bibliotecas
!pip install fuzzywuzzy  
!pip install geopandas
!pip install geopy

     |████████████████████████████████| 962kB 2.8MB/s 
     |████████████████████████████████| 14.7MB 245kB/s 
     |████████████████████████████████| 10.9MB 47.6MB/s 


**Importar bibliotecas**

In [ ]:
from pprint import pprint
import csv
import pandas as pd 
import re
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import datetime
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim

In [ ]:
#Abre banco de dados

#file = open('twitter-bombeiros-db-abril.csv', 'r', encoding='latin-1', newline='')

df = pd.read_csv("/content/drive/My Drive/python/bombeiros/twitter-bombeiros-db-atualizado-julho-2020.csv", encoding='utf-8', sep='|') 

#estes valores precisam ser substituidos pois estavam quebrando o código - pois são sinonimos e quebrava o código
df= df.replace(to_replace ='Rua SC-413 - Rodovia do Arroz', value = "Rodovia do Arroz", regex = True) 
df= df.replace(to_replace ='A guarnição da Unidade 10 – Aventureiro teve uma recepção calorosa, em atendimento a ocorrência, na tarde desde domingo, em uma residência no Parque Guarani. O pequeno João Miguel Campagnaro, além de... https://www.facebook.com/bombeirosvoluntariosdejlle/posts/3351860768219007', value = "", regex = True) 
df= df.replace(to_replace ='Em defesa da vida Os bombeiros voluntários atenderam uma ocorrência', value = "tweet descartado", regex = True) 
df= df.replace(to_replace ='Descrição da ocorrência: resgate de um brinquedo', value = "tweet descartado", regex = True) 
df= df.replace(to_replace ='ocorrência onde condutor colidiu o veículo em poste de energia', value = "tweet descartado", regex = True) 
df= df.replace(to_replace ='Ocorrência na rua Max Colin. Resgate no sexto andar de prédio em obras. #cbvj', value = "tweet descartado", regex = True) 


Abre lista de bairros de joinville para dar match

In [ ]:
#Cria uma lista com todos os bairros de Joinville
df_bairros = pd.read_csv("/content/drive/My Drive/python/bombeiros/lista-bairros-jlle.csv", encoding='latin-1', sep='|', header=None)
lista_bairros = []

for row in df_bairros[0]:
    lista_bairros.append(row)

In [ ]:
df_filtro_novo = df[df['texto'].str.contains("\d\d.\d\d .* \d\d.\d\d ", na=False)]
#df_filtro_novo

df1_filtro['texto'][230:260]

In [ ]:
df_dados = pd.DataFrame()

for index, row in df_filtro_novo.iterrows():
    # Cria dicionario para armazenar valores
    dados = {}
    dados['index'] = index
    
    #Captura e salva ano
    regex_data = re.compile('\d\d\d\d')
    dados['ano'] = regex_data.findall(row['data'])[0]
    
    # Catpura e salva Data
    regex_data = re.compile('\d\d/\d\d')
    dados['data'] = regex_data.findall(row['texto'])[0]

    # Captura e salva mês
    dados['mes'] = dados['data'].split('/')[1]
    
    # Captura e salva Hora
    regex_hora = re.compile('\d\d[h:]\d\d')
    dados["hora"] = regex_hora.findall(row['texto'])[0]

    # Captura e Salva endereço:
    try:
        regex_endereco = re.compile('(([Rr]ua|Rodovia|[Ee]strada|[Aa]venida|[Pp]raça).*)(?:-)')
        obj_endereco = regex_endereco.search(row['texto'])
        dados["endereco"] = obj_endereco.group(1)
        
        #Alguns resultados incluiam o ''-bairro'', a linha seguinte corrige isso
        if '-' in dados['endereco']:
            endereco_limpo = dados['endereco'].split('-', 1)[0]
            try:
                regex_num = re.compile('([0-9]+)')
                numero_endereco = regex_num.findall(dados["endereco"])[0]
                dados['endereco'] = endereco_limpo + ", " + numero_endereco
            except IndexError:
                dados['endereco'] = endereco_limpo
        else:
            pass    
    
    except AttributeError:
        dados["endereco"] = ""
    
    # Caputra e salva Descrição
    try:
        regex_descricao = re.compile('(?:.*) - (?:.*) - (.*) (?:(Localização:)|[Rr]ua|[Rr]odovia|[Ee]strada|[Aa]venida|[Pp]raça)')
        obj_descricao = regex_descricao.search(row['texto'])
        dados["descricao"] = obj_descricao.group(1)
        dados["descricao"] = str(dados["descricao"]).strip()
       
        #Alguns resultados incluem 'localização', esse trecho corrige isso
        if 'Localização:' in dados["descricao"]:
            dados["descricao"] = dados["descricao"].split('Localização')[0]
            dados["descricao"] = str(dados["descricao"]).strip()
        
        #Alguns resultados incluem 'Rua', esse trecho corrige isso
        elif 'Rua' in dados["descricao"]:
            dados["descricao"] = dados["descricao"].split('Rua')[0]
            dados["descricao"] = str(dados["descricao"]).strip()
        
        #Alguns resultados incluem 'Ref.', esse trecho corrige isso
        elif 'Ref.' in dados["descricao"]:
            dados["descricao"] = dados["descricao"].split('Ref.')[0]
            dados["descricao"] = str(dados["descricao"]).strip()
        
        else:
            pass
            
    except AttributeError:
        dados["descricao"] = ""
    
    #Acha o Bairro 
    str2Match = row['texto']
    strOptions = lista_bairros
    highest = process.extractOne(str2Match,strOptions)
    dados["bairro"] = highest[0]
    
    
    #Remove espaços em branco no começo e fim do dicionário
    for k,v in dados.items():
        if isinstance(v, str) == True:
            v = str(v).strip()
        else:
            pass
    
    #Adiciona dados a uma linha de tabela provisória
    df1  = pd.DataFrame(dados, index=[dados['index']])
    
    #Armazena valores na tabela de output
    df_dados = pd.concat([df_dados, df1])

    #Imprime o index, linha da tabela e nova linha para acompanhar
    print( "### O index é: " + str(index) + " ####")
    print(dados)
    print("\n")

Confere se a tabela final ficou boa

In [ ]:
df_dados

,index,ano,data,mes,hora,endereco,descricao,bairro
0,0,2020,13/07,07,19h15,Rua Marcos Joao Serafim,Carro x Moto,Nova Brasília
1,1,2020,13/07,07,18h59,"Rua Luiz Bachtold, 596",Carro x Moto,Costa e Silva
4,4,2020,13/07,07,15h49,"Rua Estrada Mildau, 775",Queda de Nível,Centro
5,5,2020,13/07,07,15h40,"Rua Arco, 169",Queda de Nível,Jardim Iririú
6,6,2020,13/07,07,14h33,"Rua Estrada Oeste, 5319",Queda de Bicicleta,Centro
...,...,...,...,...,...,...,...,...
2995,2995,2020,05/02,02,08h51,"Rua Inambu, 2949",Queda de Nível,Costa e Silva
2996,2996,2020,05/02,02,08h39,Rua Albano Schmidt,Emergência Médica/Clínico,Boa Vista
2997,2997,2020,05/02,02,07h59,"Rodovia Sc, 108",Emergência Médica/Clínico,Vila Nova
2998,2998,2020,05/02,02,07h12,"Rua Professor Clemens Schmidt, 56",Corte e/ou Poda de Árvore,Floresta



**Adiciona coordenadas**

In [ ]:
#cria cópia de planilha e cria coluna endereço completo
df_endereco = df_dados.copy()
for index, row in df_endereco.iterrows():
  df_endereco.loc[index, "endereco_completo"] = str(row['endereco']) + ", " + str(row['bairro']) + ", Joinville, Santa Catarina, Brazil"

In [ ]:
#se estiver de bom humor, pega latitude e longitude
df_cord = df_endereco.copy()
for index, row in df_endereco.iterrows():
  try:
    locator = Nominatim(user_agent='myGeocoder')
    location = locator.geocode(row['endereco_completo'])
    df.loc[index, "latitude"] = location.latitude
    df.loc[index, "longitude"] = location.longitude
    print(str(index) + ": " + "--Coordenadas: " + str(location.latitude) + " " + str(location.longitude))
  except (AttributeError, KeyError, TypeError) as e:
    print(row['endereco_completo'] + ": ### Falhou")
    pass
  
  

Rua Marcos Joao Serafim , Nova Brasília, Joinville, Santa Catarina, Brazil: ### Falhou
1: --Coordenadas: -26.265673 -48.8817475
Rua Estrada Mildau, 775 , Centro, Joinville, Santa Catarina, Brazil: ### Falhou
Rua Arco, 169, Jardim Iririú, Joinville, Santa Catarina, Brazil: ### Falhou
Rua Estrada Oeste, 5319 , Centro, Joinville, Santa Catarina, Brazil: ### Falhou
8: --Coordenadas: -26.369112 -48.8327736
9: --Coordenadas: -26.331693 -48.8695828
10: --Coordenadas: -26.327678 -48.8076252
11: --Coordenadas: -26.2286797 -48.8053199
Rua Estrada do Oeste , Morro do Meio, Joinville, Santa Catarina, Brazil: ### Falhou
13: --Coordenadas: -26.2797054 -48.8651456
14: --Coordenadas: -26.3379569 -48.7911511
15: --Coordenadas: -26.335977 -48.8507188
Rua 1111 , 1111, Centro, Joinville, Santa Catarina, Brazil: ### Falhou
18: --Coordenadas: -26.337631 -48.8423012
19: --Coordenadas: -26.3387646 -48.8254677
20: --Coordenadas: -26.2806049 -48.7967897
21: --Coordenadas: -26.2577551 -48.8122235
22: --Coordenad

Une os dados limpos à tabela original

In [ ]:
#Junta tabela com dados limpos com tabela original
df_unido = pd.concat([df, df_dados], axis=1)

#unifica tabelas
df_completo = pd.concat([df_unido, df_cord['endereco_completo']], axis=1, sort=False)
df_completo


#remove valdores duplicados de tweets iguais
df_final = df_completo.drop_duplicates(subset ="texto", keep='first')    

df_final


,data,texto,mentions,hashtags,favoritos,retweets,to,tweetId,link,latitude,longitude,index,ano,data,mes,hora,endereco,descricao,bairro,endereco_completo
0,13-07-2020 - 22:18:20,Ocorrência: 13/07 - 19h15 - Carro x Moto Local...,NaN,NaN,1,0,NaN,1282801564063866880,https://twitter.com/bvsc_joinville/status/1282...,NaN,NaN,0.0,2020,13/07,07,19h15,Rua Marcos Joao Serafim,Carro x Moto,Nova Brasília,"Rua Marcos Joao Serafim , Nova Brasília, Joinv..."
1,13-07-2020 - 22:00:21,Ocorrência: 13/07 - 18h59 - Carro x Moto Local...,NaN,NaN,0,0,NaN,1282797036685922304,https://twitter.com/bvsc_joinville/status/1282...,-26.265673,-48.881748,1.0,2020,13/07,07,18h59,"Rua Luiz Bachtold, 596",Carro x Moto,Costa e Silva,"Rua Luiz Bachtold, 596 , Costa e Silva, Joinvi..."
2,13-07-2020 - 19:18:40,#livedosdosbombeirosvoluntarios,NaN,#livedosdosbombeirosvoluntarios,3,0,NaN,1282756346568376320,https://twitter.com/bvsc_joinville/status/1282...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13-07-2020 - 19:09:34,"Como ocorre todos os anos, a comunidade demon...",NaN,NaN,2,0,NaN,1282754058605002752,https://twitter.com/bvsc_joinville/status/1282...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13-07-2020 - 18:49:20,Ocorrência: 13/07 - 15h49 - Queda de Nível Loc...,NaN,NaN,1,0,NaN,1282748965289635840,https://twitter.com/bvsc_joinville/status/1282...,NaN,NaN,4.0,2020,13/07,07,15h49,"Rua Estrada Mildau, 775",Queda de Nível,Centro,"Rua Estrada Mildau, 775 , Centro, Joinville, S..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,05-02-2020 - 12:00:56,Ocorrência: 05/02 - 08h51 - Queda de Nível Loc...,NaN,NaN,1,0,NaN,1225026520198078464,https://twitter.com/bvsc_joinville/status/1225...,-26.273515,-48.882223,2995.0,2020,05/02,02,08h51,"Rua Inambu, 2949",Queda de Nível,Costa e Silva,"Rua Inambu, 2949 , Costa e Silva, Joinville, S..."
2996,05-02-2020 - 11:51:55,Ocorrência: 05/02 - 08h39 - Emergência Médica/...,NaN,NaN,1,0,NaN,1225024249997512706,https://twitter.com/bvsc_joinville/status/1225...,-26.310441,-48.830077,2996.0,2020,05/02,02,08h39,Rua Albano Schmidt,Emergência Médica/Clínico,Boa Vista,"Rua Albano Schmidt , Boa Vista, Joinville, San..."
2997,05-02-2020 - 11:08:56,Ocorrência: 05/02 - 07h59 - Emergência Médica/...,NaN,NaN,1,0,NaN,1225013433977970690,https://twitter.com/bvsc_joinville/status/1225...,NaN,NaN,2997.0,2020,05/02,02,07h59,"Rodovia Sc, 108",Emergência Médica/Clínico,Vila Nova,"Rodovia Sc, 108, Vila Nova, Joinville, Santa C..."
2998,05-02-2020 - 10:20:53,Ocorrência: 05/02 - 07h12 - Corte e/ou Poda de...,NaN,NaN,1,0,NaN,1225001339870203905,https://twitter.com/bvsc_joinville/status/1225...,-26.344029,-48.848707,2998.0,2020,05/02,02,07h12,"Rua Professor Clemens Schmidt, 56",Corte e/ou Poda de Árvore,Floresta,"Rua Professor Clemens Schmidt, 56 , Floresta, ..."


Salva o arquivo final

In [ ]:
df_final.to_csv('bombeiros_limpo_regex_15_julho.csv', sep="|",index=False)
!cp bombeiros_limpo_regex_15_julho.csv "/content/drive/My Drive/python/bombeiros"

#**EXTRA**: Une as dutas tabelas de de tweets e cria uma atualizada


In [ ]:
###### EXTRA  ##### Une as dutas tabelas de de tweets e cria uma atualizada

df_3 = pd.read_csv('/content/drive/My Drive/python/bombeiros/twitter-bombeiros-db-maio.csv', sep='|') 
df_4 = pd.read_csv('/content/drive/My Drive/python/bombeiros/twitter-bombeiros-db-atualizado-junho-2020.csv', sep='|') 

df_unida = pd.concat([df_4, df_3], axis=0, sort=True)

#remove os mesmos tweets
df_tweetid_unico = df_unida.drop_duplicates(subset='tweetId',keep='first')

#para garantir, tira também os tweets com mesmo texto
df_final = df_tweetid_unico.drop_duplicates(subset='texto',keep='first')

print("O numero de da base original é de " + str(df_3.shape))
print("O numero de da base suja é de " + str(df_unida.shape))
print("O numero da base com tweetid unico é " + str(df_tweetid_unico.shape))
print("O numero da base com texto uniico é  " + str(df_final.shape))

df_final

out_name = "twitter-bombeiros-db-junho.csv"
df_final.to_csv(out_name, sep="|", index=False)

O numero de da base original é de (20231, 9)
O numero de da base suja é de (23231, 9)
O numero da base com tweetid unico é (20838, 9)
O numero da base com texto uniico é  (20832, 9)


Copia arquivo para o Drive

In [ ]:
!cp twitter-bombeiros-db-junho.csv "/content/drive/My Drive/python/bombeiros"